# Encoding a neural network generated with Julia into a JSON file

- Step 1: generate a non-linear dataset
- Step 2: Write data to a CSV file
- Step 3: Build the neural network
- Step 4: Train the neural network
- Step 5: Encode NN to JSON file

In [1]:
# imports
using Pkg; Pkg.activate(".")

using Flux, Plots, Random, Statistics, LaplaceRedux
using Flux.Optimise: update!, Adam
theme(:lime)

using CSV
using DataFrames

using JSON
using Serialization
using Tullio

using LinearAlgebra
using Zygote

Random.seed!(42)

  Activating new project at `c:\Users\adeli\OneDrive\Desktop\facultate\2nd year\Q4 - Software Project\LaplaceRedux.jl\dev\notebooks\two-class`


TaskLocalRNG()

### Step 1: generate a non-linear dataset

In [77]:
xs, ys = LaplaceRedux.Data.toy_data_multi(50)
X = hcat(xs...) # bring into tabular format
data = zip(xs,ys)

zip([[1.6801393795620563, 3.5263544599281134], [4.363742277837995, 2.6917177869381694], [4.532470758397958, 5.30055885839391], [4.530641576744509, 3.2990163059418425], [3.5734780996754285, 3.4242794223236332], [4.449988593431152, 5.100834667776773], [3.2926827627045308, 3.727440437758293], [1.4481509071599454, 5.3609283716244684], [1.1374273782015822, 3.924623406623718], [1.924779917904933, 3.8171052896062365]  …  [-3.9146176308560983, 5.370221049178135], [-2.8753043050807747, 5.17113262441957], [-4.972008358231747, 1.443881045144713], [-6.325299124070565, 4.255017998387671], [-3.9904034595383386, 2.3492050148489825], [-2.8946158215244315, 1.3070612432912136], [-6.8073184865731085, 2.902228473095165], [-5.1149402113688645, 2.3935217156788036], [-5.717635911186631, 1.0902405592489193], [-2.6799715829629784, 1.5350155638884901]], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0])

### Step 2: Write data to a CSV file

In [78]:
# Open the CSV file for writing
file = "data.csv"
csv_file = open(file, "w")

# Write the header
write(csv_file, "x1,x2,y\n")

# Write the data
for ((x1, x2), y) in data
    write(csv_file, "$x1,$x2,$y\n")
end

# Close the CSV file
close(csv_file)

### Step 3: Build the neural network

In [79]:
n_hidden = 15
D = size(X,1)
nn = Chain(
    Dense(D, n_hidden, σ),
    Dense(n_hidden, length(unique(ys)))
)
loss(x, y) = Flux.Losses.logitbinarycrossentropy(nn(x), y) 

loss (generic function with 1 method)

### Step 4: Training the neural network

In [80]:
opt = Adam(1e-3)
epochs = 100
avg_loss(data) = mean(map(d -> loss(d[1],d[2]), data))
show_every = epochs/10

for epoch = 1:epochs
  for d in data
    gs = gradient(Flux.params(nn)) do
      l = loss(d...)
    end
    update!(opt, Flux.params(nn), gs)
  end
  if epoch % show_every == 0
    println("Epoch " * string(epoch))
    @show avg_loss(data)
  end
end

Epoch 10
avg_loss(data) = -5.322127319086576


Epoch 20
avg_loss(data) = -13.961049915297181
Epoch 30
avg_loss(data) = -24.72785834289505
Epoch 40
avg_loss(data) = -35.44095220747735
Epoch 50
avg_loss(data) = -45.80643894855723
Epoch 60
avg_loss(data) = -55.943613144858205


Epoch 70
avg_loss(data) = -65.90990911008669
Epoch 80
avg_loss(data) = -75.74368146738935
Epoch 90
avg_loss(data) = -85.47325754155388
Epoch 100
avg_loss(data) = -95.12010918060746


### Step 5: Encode NN to JSON file

In [81]:
serialize_json_nn(nn::Chain)::String = JSON.json([Dict(:weight => nn.layers[i].weight, :bias => nn.layers[i].bias) for i in range(1, length(nn.layers))])
# Export as JSON
write("nn.json", serialize_json_nn(nn))
serialize("nn-binary.jlb", nn)

In [82]:
# Read the JSON file
json_string = read("nn.json", String)
parsed_data = JSON.parse(json_string)

# Convert JSON string back to neural network
nndes = Chain(map(layer -> Dense(convert(Matrix{Float64}, layer["weight"]), convert(Array{Float64, 1}, layer["bias"])), parsed_data))

println(nn)
println(nndes)

MethodError: MethodError: no method matching Matrix{Float64}(::Vector{Any})
Closest candidates are:
  Array{T, N}(::AbstractArray{S, N}) where {T, N, S} at array.jl:626
  Array{T, N}(!Matched::Union{Union{NNlib.BatchedAdjoint{T, <:CUDA.CuArray{T}}, NNlib.BatchedTranspose{T, <:CUDA.CuArray{T}}} where T, Union{Base.LogicalIndex{T, <:Union{NNlib.BatchedAdjoint{T, <:CUDA.CuArray{T}}, NNlib.BatchedTranspose{T, <:CUDA.CuArray{T}}}}, Base.ReinterpretArray{T, N, <:Any, <:Union{SubArray{<:Any, <:Any, var"#s14"}, var"#s14"}} where var"#s14"<:Union{NNlib.BatchedAdjoint{T, <:CUDA.CuArray{T}}, NNlib.BatchedTranspose{T, <:CUDA.CuArray{T}}}, Base.ReshapedArray{T, N, <:Union{Base.ReinterpretArray{<:Any, <:Any, <:Any, <:Union{SubArray{<:Any, <:Any, var"#s15"}, var"#s15"}}, SubArray{<:Any, <:Any, var"#s15"}, var"#s15"}} where var"#s15"<:Union{NNlib.BatchedAdjoint{T, <:CUDA.CuArray{T}}, NNlib.BatchedTranspose{T, <:CUDA.CuArray{T}}}, SubArray{T, N, <:Union{Base.ReinterpretArray{<:Any, <:Any, <:Any, <:Unio